In [1]:
import tensorflow as tf 
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
%matplotlib inline

c:\users\you\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\you\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\you\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\you\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type

#### 数据集

In [2]:
def load_data_from_xlsx(path="./src_data.xlsx",all_features=False):
    """
    Load data from file and decide whether 4 features or all features are kept. 
    """
    data = pd.read_excel(path)
    if not all_features:
        keep_features = ["A_NIH", "A_THALAMUS2", "MIDDLE_BAO", "BATMAN", "FR"]
        data = data.drop(
            [feature for feature in data.columns if feature not in keep_features],
            axis=1,
        )
    print("Data loaded from xlsx.")
    return data

In [3]:
def one_hot_encode(data,categoricals=["A_THALAMUS2", "MIDDLE_BAO", "BATMAN"]):
    """One-hot encode categorical data"""
    for column in categoricals:
        data = pd.get_dummies(data, columns=[column], prefix=[column])
    
    print("One-hot encode categorical data.")
    return data

In [4]:
def scale(X_train):
    """Scale independent variables"""
    X_scaler = preprocessing.StandardScaler()

    X_train_scaled = X_scaler.fit_transform(X_train.values.astype(float))

    return [X_train_scaled,X_scaler]

In [5]:
def split_pipeline(data, output='FR',test_size=.2):
    """
    Split data into variables
    Arguments: Pandas dataframe, output column (dependent variable),size of test set(account for all data)
    Returns: List of scaled and unscaled dependent and independent variables
    """
    y, X = data.iloc[:, -1], data.iloc[:, :-1]
    X_train, X_test, y_train, y_test = train_test_split(
        data.drop([output], axis=1), data[output], test_size=test_size, random_state=1)
    [X_train_scaled,X_scaler] = scale(X_train)
    
    enc = preprocessing.OneHotEncoder()
    y_train = enc.fit_transform(np.array(y_train).reshape(-1,1)).toarray()
    y_test = enc.fit_transform(np.array(y_test).reshape(-1,1)).toarray()
    return [np.array(X_train_scaled),y_train,np.array(X_test),y_test,X_scaler]

In [6]:
class Recording():
    '''
    Define a dataframe used to record every experiment
    Arguments:parameters to be recored
    '''
    def __init__(self,index=None): 
        if not index:
            self.index = (
            [
                "training_epochs",
                "batch_size",
                "is_lr_decay",
                "init_learning_rate",
                "decay_rate",
                "decay_steps",
                "num_hidden_layers",
                "num_units_each_layer",
                "is_dropout",
                "dropout_rate",
            ]
            + ["accuracy_" + str(index) for index in range(1, 11)]
            + ["auc_" + str(index) for index in range(1, 11)]
            + ["avg_accuracy", "avg_auc"]
        )
        else:
            self.index = index
    def getIndex(self):
        return self.index
    def getRecordFile(self):
        return pd.DataFrame(columns=self.index)
    

In [7]:
def data_pipeline():
    data = load_data_from_xlsx()
    data = one_hot_encode(data)
    X_train_scaled, y_train, X_test, y_test, X_scaler = split_pipeline(data)
    print("Training set and test set are generated.")
    return X_train_scaled, y_train, X_test, y_test, X_scaler

#### Define Network

In [8]:
# Machine Learning-Based Model for Prediction of Outcomes in Acute Stroke
# 3 hidden layers with 15 artificial neural network units each were used

tf.set_random_seed(0)

# parameters
init_learning_rate = 0.006
training_epochs = 600
batch_size = 10
display_step = 10
decay_rate = 0.96 
decay_steps = 50 
use_learning_decay = True
use_dropout = True

# Networks Parameters
X_train_scaled,y_train,X_test,y_test,X_scaler = data_pipeline() # here just be used to initialize two parameters of 'dim' and 'nclass' 
dim = X_train_scaled.shape[1]
nclass = y_train.shape[1]
n_hidden_1 = 15
n_hidden_2 = 15
n_hidden_3 = 15
n_input = dim
n_classes = nclass

# tf.Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])
dropout_keep_prob = tf.placeholder('float')
learning_rate = tf.placeholder('float')
global_step = tf.Variable(tf.constant(0))
lr = tf.train.exponential_decay(init_learning_rate,global_step,decay_steps,decay_rate)

# Store layers weight & bias
stddev = 0.1  # important
weights = {
    "h1": tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev)),
    "h2": tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
    "h3": tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3], stddev=stddev)),
    "out": tf.Variable(tf.random_normal([n_hidden_3, n_classes], stddev=stddev))
}

biases = {
    "b1": tf.Variable(tf.random_normal([n_hidden_1])),
    "b2": tf.Variable(tf.random_normal([n_hidden_2])),
    "b3": tf.Variable(tf.random_normal([n_hidden_3])),
    "out": tf.Variable(tf.random_normal([n_classes])),
}

# Create model
def multiplayer_perception(_X, _weights, _biases,_keep_prob):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(_X, _weights["h1"]), _biases["b1"]))  
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, _weights["h2"]), _biases["b2"]))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, _weights["h3"]), _biases["b3"]))
    if use_dropout:
        layer_3 = tf.nn.dropout(layer_3,_keep_prob)
    out = tf.add(tf.matmul(layer_3, _weights["out"]) , _biases["out"]) 
    return out
print("Network Ready!")

Data loaded from xlsx.
One-hot encode categorical data.
Training set and test set are generated.
Network Ready!


#### Define Functions

In [9]:
# Construct Model
pred = multiplayer_perception(x,weights,biases,dropout_keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=pred,labels=y))
optm = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))    
accu = tf.reduce_mean(tf.cast(corr, "float"))

# Initializing the variables

print ("Functions ready")

Functions ready


#### Optimize

In [10]:
def train(X_train_scaled,y_train,X_test,y_test,X_scaler):
    ntrain = y_train.shape[0]
    ntest = y_test.shape[0]
    # Launch the graph
    init = tf.initialize_all_variables()
    sess = tf.Session()
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        if use_learning_decay:
            lr_ = sess.run(lr,feed_dict={global_step:epoch})
        else:
            lr_ = init_learning_rate
        avg_cost = 0.
        total_batch = int(ntrain/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            randidx = np.random.randint(ntrain, size=batch_size)
            batch_xs = X_train_scaled[randidx, :]
            batch_ys = y_train[randidx, :]
            # Fit training using batch data
            if use_dropout:
                sess.run(optm, feed_dict={x: batch_xs, y: batch_ys,dropout_keep_prob:0.8,learning_rate:lr_})
            else:
                sess.run(optm, feed_dict={x: batch_xs, y: batch_ys,dropout_keep_prob:1.,learning_rate:lr_})
            # Compute average loss
            avg_cost += sess.run(cost, 
                    feed_dict={x: batch_xs, y: batch_ys,dropout_keep_prob:1.0})/total_batch
            # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print ("Epoch: %03d/%03d ,learning_rate:%.6f,cost: %.9f" % 
                   (epoch+1, training_epochs, lr_,avg_cost))
            train_acc = sess.run(accu, feed_dict={x: X_train_scaled, y: y_train,dropout_keep_prob:1.0})
            print (" Training accuracy: %.3f" % (train_acc))
            test_acc,auc = evaluate(sess,X_test,y_test,X_scaler)
            print (" Test accuracy: %.3f" % (test_acc))
            print (" Test ROC_AUC: %.3f" % (auc))
    print ("Optimization Finished!")
    test_acc,auc = evaluate(sess,X_test,y_test,X_scaler)
    sess.close()
    return test_acc , auc

In [11]:
def evaluate(sess,X_test,y_test,X_scaler):
    X_test_scaled = X_scaler.transform(X_test.astype(float))
    test_acc = sess.run(accu, feed_dict={x: X_test_scaled, y: y_test, dropout_keep_prob: 1.0})
    probs = sess.run(tf.sigmoid(pred),feed_dict={x: X_test_scaled, y: y_test,dropout_keep_prob:1.0})[:,1]
    auc = roc_auc_score(np.argmax(y_test,1),probs)
    return [test_acc,auc]

In [12]:
def experiments_and_recording():
    rec = Recording()
    record = rec.getRecordFile()
    accuracy_10_runs = []
    auc_10_runs = []
    one_10_runs = [
                    training_epochs,
                    batch_size,
                    False,
                    init_learning_rate,
                    decay_rate,
                    decay_steps,
                    3,
                    (n_hidden_1,n_hidden_2,n_hidden_3),
                    False,
                    1
                ]
    for i in range(10):
        X_train_scaled,y_train,X_test,y_test,X_scaler = data_pipeline()
        acc,auc = train(X_train_scaled,y_train,X_test,y_test,X_scaler)
        accuracy_10_runs.append(acc)
        auc_10_runs.append(auc)
    one_10_runs.extend(accuracy_10_runs)
    one_10_runs.extend(auc_10_runs)
    one_10_runs.append(np.mean(accuracy_10_runs))
    one_10_runs.append(np.mean(auc_10_runs))
    record = record.append(pd.Series(one_10_runs,index=rec.getIndex()),ignore_index=True)
    record.to_csv('record.csv')


In [13]:
experiments_and_recording()

Data loaded from xlsx.
One-hot encode categorical data.
Training set and test set are generated.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 010/600 ,learning_rate:0.005956,cost: 0.692171282
 Training accuracy: 0.565
 Test accuracy: 0.565
 Test ROC_AUC: 0.769
Epoch: 020/600 ,learning_rate:0.005908,cost: 0.388295702
 Training accuracy: 0.826
 Test accuracy: 0.739
 Test ROC_AUC: 0.731
Epoch: 030/600 ,learning_rate:0.005860,cost: 0.581687470
 Training accuracy: 0.837
 Test accuracy: 0.696
 Test ROC_AUC: 0.723
Epoch: 040/600 ,learning_rate:0.005812,cost: 0.425460077
 Training accuracy: 0.848
 Test accuracy: 0.652
 Test ROC_AUC: 0.708
Epoch: 050/600 ,learning_rate:0.005765,cost: 0.385052158
 Training accuracy: 0.859
 Test accuracy: 0.696
 Test ROC_AUC: 0.700
Epoch: 060/600 ,learning_rate:0.005718,cost: 0.397766183
 Training accuracy: 0.859
 Test accuracy: 0.652
 Test ROC_AUC: 0.700
Epoch: 070/600 ,learning_rate:0.005671,cost: 0.476657574
 Training accura

Epoch: 030/600 ,learning_rate:0.005860,cost: 0.482230292
 Training accuracy: 0.837
 Test accuracy: 0.652
 Test ROC_AUC: 0.715
Epoch: 040/600 ,learning_rate:0.005812,cost: 0.380537131
 Training accuracy: 0.848
 Test accuracy: 0.609
 Test ROC_AUC: 0.708
Epoch: 050/600 ,learning_rate:0.005765,cost: 0.377529797
 Training accuracy: 0.848
 Test accuracy: 0.652
 Test ROC_AUC: 0.708
Epoch: 060/600 ,learning_rate:0.005718,cost: 0.450280362
 Training accuracy: 0.837
 Test accuracy: 0.652
 Test ROC_AUC: 0.715
Epoch: 070/600 ,learning_rate:0.005671,cost: 0.484935237
 Training accuracy: 0.848
 Test accuracy: 0.696
 Test ROC_AUC: 0.708
Epoch: 080/600 ,learning_rate:0.005625,cost: 0.387765245
 Training accuracy: 0.826
 Test accuracy: 0.652
 Test ROC_AUC: 0.708
Epoch: 090/600 ,learning_rate:0.005579,cost: 0.351413361
 Training accuracy: 0.848
 Test accuracy: 0.652
 Test ROC_AUC: 0.708
Epoch: 100/600 ,learning_rate:0.005534,cost: 0.391907521
 Training accuracy: 0.859
 Test accuracy: 0.652
 Test ROC_AUC

Epoch: 090/600 ,learning_rate:0.005579,cost: 0.441374956
 Training accuracy: 0.826
 Test accuracy: 0.652
 Test ROC_AUC: 0.708
Epoch: 100/600 ,learning_rate:0.005534,cost: 0.478664160
 Training accuracy: 0.859
 Test accuracy: 0.609
 Test ROC_AUC: 0.700
Epoch: 110/600 ,learning_rate:0.005489,cost: 0.376377319
 Training accuracy: 0.826
 Test accuracy: 0.652
 Test ROC_AUC: 0.723
Epoch: 120/600 ,learning_rate:0.005444,cost: 0.359221768
 Training accuracy: 0.826
 Test accuracy: 0.652
 Test ROC_AUC: 0.731
Epoch: 130/600 ,learning_rate:0.005400,cost: 0.328588315
 Training accuracy: 0.848
 Test accuracy: 0.696
 Test ROC_AUC: 0.754
Epoch: 140/600 ,learning_rate:0.005356,cost: 0.328982136
 Training accuracy: 0.848
 Test accuracy: 0.652
 Test ROC_AUC: 0.746
Epoch: 150/600 ,learning_rate:0.005313,cost: 0.297934903
 Training accuracy: 0.826
 Test accuracy: 0.652
 Test ROC_AUC: 0.746
Epoch: 160/600 ,learning_rate:0.005270,cost: 0.419380761
 Training accuracy: 0.815
 Test accuracy: 0.652
 Test ROC_AUC

 Test accuracy: 0.652
 Test ROC_AUC: 0.746
Epoch: 150/600 ,learning_rate:0.005313,cost: 0.362521898
 Training accuracy: 0.815
 Test accuracy: 0.652
 Test ROC_AUC: 0.754
Epoch: 160/600 ,learning_rate:0.005270,cost: 0.236383134
 Training accuracy: 0.848
 Test accuracy: 0.652
 Test ROC_AUC: 0.762
Epoch: 170/600 ,learning_rate:0.005227,cost: 0.315536857
 Training accuracy: 0.848
 Test accuracy: 0.652
 Test ROC_AUC: 0.777
Epoch: 180/600 ,learning_rate:0.005184,cost: 0.275520030
 Training accuracy: 0.826
 Test accuracy: 0.652
 Test ROC_AUC: 0.777
Epoch: 190/600 ,learning_rate:0.005142,cost: 0.338805305
 Training accuracy: 0.837
 Test accuracy: 0.652
 Test ROC_AUC: 0.769
Epoch: 200/600 ,learning_rate:0.005100,cost: 0.266795170
 Training accuracy: 0.804
 Test accuracy: 0.652
 Test ROC_AUC: 0.785
Epoch: 210/600 ,learning_rate:0.005059,cost: 0.283335532
 Training accuracy: 0.815
 Test accuracy: 0.652
 Test ROC_AUC: 0.785
Epoch: 220/600 ,learning_rate:0.005018,cost: 0.309069398
 Training accuracy

Epoch: 190/600 ,learning_rate:0.005142,cost: 0.402451995
 Training accuracy: 0.826
 Test accuracy: 0.696
 Test ROC_AUC: 0.792
Epoch: 200/600 ,learning_rate:0.005100,cost: 0.299640233
 Training accuracy: 0.815
 Test accuracy: 0.652
 Test ROC_AUC: 0.800
Epoch: 210/600 ,learning_rate:0.005059,cost: 0.273758639
 Training accuracy: 0.804
 Test accuracy: 0.652
 Test ROC_AUC: 0.808
Epoch: 220/600 ,learning_rate:0.005018,cost: 0.352827700
 Training accuracy: 0.804
 Test accuracy: 0.652
 Test ROC_AUC: 0.800
Epoch: 230/600 ,learning_rate:0.004977,cost: 0.293558978
 Training accuracy: 0.815
 Test accuracy: 0.696
 Test ROC_AUC: 0.808
Epoch: 240/600 ,learning_rate:0.004936,cost: 0.283647526
 Training accuracy: 0.804
 Test accuracy: 0.696
 Test ROC_AUC: 0.815
Epoch: 250/600 ,learning_rate:0.004896,cost: 0.342406117
 Training accuracy: 0.804
 Test accuracy: 0.696
 Test ROC_AUC: 0.815
Epoch: 260/600 ,learning_rate:0.004856,cost: 0.309843643
 Training accuracy: 0.804
 Test accuracy: 0.696
 Test ROC_AUC

Epoch: 250/600 ,learning_rate:0.004896,cost: 0.222591715
 Training accuracy: 0.793
 Test accuracy: 0.696
 Test ROC_AUC: 0.800
Epoch: 260/600 ,learning_rate:0.004856,cost: 0.261997094
 Training accuracy: 0.826
 Test accuracy: 0.696
 Test ROC_AUC: 0.808
Epoch: 270/600 ,learning_rate:0.004817,cost: 0.287675255
 Training accuracy: 0.815
 Test accuracy: 0.696
 Test ROC_AUC: 0.792
Epoch: 280/600 ,learning_rate:0.004778,cost: 0.277080727
 Training accuracy: 0.837
 Test accuracy: 0.696
 Test ROC_AUC: 0.800
Epoch: 290/600 ,learning_rate:0.004739,cost: 0.280220075
 Training accuracy: 0.815
 Test accuracy: 0.696
 Test ROC_AUC: 0.792
Epoch: 300/600 ,learning_rate:0.004700,cost: 0.321459818
 Training accuracy: 0.815
 Test accuracy: 0.696
 Test ROC_AUC: 0.800
Epoch: 310/600 ,learning_rate:0.004662,cost: 0.302616109
 Training accuracy: 0.804
 Test accuracy: 0.696
 Test ROC_AUC: 0.800
Epoch: 320/600 ,learning_rate:0.004624,cost: 0.289737880
 Training accuracy: 0.804
 Test accuracy: 0.696
 Test ROC_AUC

Epoch: 310/600 ,learning_rate:0.004662,cost: 0.331377702
 Training accuracy: 0.826
 Test accuracy: 0.652
 Test ROC_AUC: 0.777
Epoch: 320/600 ,learning_rate:0.004624,cost: 0.311867707
 Training accuracy: 0.859
 Test accuracy: 0.696
 Test ROC_AUC: 0.785
Epoch: 330/600 ,learning_rate:0.004587,cost: 0.364611433
 Training accuracy: 0.826
 Test accuracy: 0.696
 Test ROC_AUC: 0.785
Epoch: 340/600 ,learning_rate:0.004549,cost: 0.248978813
 Training accuracy: 0.837
 Test accuracy: 0.696
 Test ROC_AUC: 0.785
Epoch: 350/600 ,learning_rate:0.004512,cost: 0.291513437
 Training accuracy: 0.826
 Test accuracy: 0.652
 Test ROC_AUC: 0.785
Epoch: 360/600 ,learning_rate:0.004476,cost: 0.258756151
 Training accuracy: 0.848
 Test accuracy: 0.696
 Test ROC_AUC: 0.785
Epoch: 370/600 ,learning_rate:0.004439,cost: 0.279770941
 Training accuracy: 0.859
 Test accuracy: 0.696
 Test ROC_AUC: 0.785
Epoch: 380/600 ,learning_rate:0.004403,cost: 0.181853784
 Training accuracy: 0.826
 Test accuracy: 0.696
 Test ROC_AUC

Epoch: 360/600 ,learning_rate:0.004476,cost: 0.312139979
 Training accuracy: 0.848
 Test accuracy: 0.696
 Test ROC_AUC: 0.823
Epoch: 370/600 ,learning_rate:0.004439,cost: 0.217003297
 Training accuracy: 0.848
 Test accuracy: 0.696
 Test ROC_AUC: 0.823
Epoch: 380/600 ,learning_rate:0.004403,cost: 0.224003817
 Training accuracy: 0.826
 Test accuracy: 0.696
 Test ROC_AUC: 0.823
Epoch: 390/600 ,learning_rate:0.004367,cost: 0.290092185
 Training accuracy: 0.826
 Test accuracy: 0.696
 Test ROC_AUC: 0.823
Epoch: 400/600 ,learning_rate:0.004332,cost: 0.272757336
 Training accuracy: 0.815
 Test accuracy: 0.696
 Test ROC_AUC: 0.823
Epoch: 410/600 ,learning_rate:0.004297,cost: 0.294481471
 Training accuracy: 0.826
 Test accuracy: 0.696
 Test ROC_AUC: 0.823
Epoch: 420/600 ,learning_rate:0.004262,cost: 0.264771764
 Training accuracy: 0.815
 Test accuracy: 0.696
 Test ROC_AUC: 0.823
Epoch: 430/600 ,learning_rate:0.004227,cost: 0.348687741
 Training accuracy: 0.826
 Test accuracy: 0.739
 Test ROC_AUC

Epoch: 420/600 ,learning_rate:0.004262,cost: 0.227250052
 Training accuracy: 0.848
 Test accuracy: 0.609
 Test ROC_AUC: 0.746
Epoch: 430/600 ,learning_rate:0.004227,cost: 0.307830163
 Training accuracy: 0.848
 Test accuracy: 0.609
 Test ROC_AUC: 0.746
Epoch: 440/600 ,learning_rate:0.004193,cost: 0.257529858
 Training accuracy: 0.848
 Test accuracy: 0.609
 Test ROC_AUC: 0.731
Epoch: 450/600 ,learning_rate:0.004159,cost: 0.258454771
 Training accuracy: 0.848
 Test accuracy: 0.609
 Test ROC_AUC: 0.738
Epoch: 460/600 ,learning_rate:0.004125,cost: 0.253300661
 Training accuracy: 0.848
 Test accuracy: 0.609
 Test ROC_AUC: 0.754
Epoch: 470/600 ,learning_rate:0.004091,cost: 0.245095592
 Training accuracy: 0.848
 Test accuracy: 0.609
 Test ROC_AUC: 0.746
Epoch: 480/600 ,learning_rate:0.004058,cost: 0.312678693
 Training accuracy: 0.848
 Test accuracy: 0.609
 Test ROC_AUC: 0.731
Epoch: 490/600 ,learning_rate:0.004025,cost: 0.269198218
 Training accuracy: 0.848
 Test accuracy: 0.609
 Test ROC_AUC

Epoch: 480/600 ,learning_rate:0.004058,cost: 0.226548795
 Training accuracy: 0.815
 Test accuracy: 0.696
 Test ROC_AUC: 0.800
Epoch: 490/600 ,learning_rate:0.004025,cost: 0.213117175
 Training accuracy: 0.815
 Test accuracy: 0.739
 Test ROC_AUC: 0.815
Epoch: 500/600 ,learning_rate:0.003992,cost: 0.268115323
 Training accuracy: 0.837
 Test accuracy: 0.696
 Test ROC_AUC: 0.808
Epoch: 510/600 ,learning_rate:0.003960,cost: 0.215212072
 Training accuracy: 0.815
 Test accuracy: 0.739
 Test ROC_AUC: 0.831
Epoch: 520/600 ,learning_rate:0.003928,cost: 0.262660324
 Training accuracy: 0.848
 Test accuracy: 0.696
 Test ROC_AUC: 0.800
Epoch: 530/600 ,learning_rate:0.003896,cost: 0.230258171
 Training accuracy: 0.837
 Test accuracy: 0.696
 Test ROC_AUC: 0.800
Epoch: 540/600 ,learning_rate:0.003864,cost: 0.223184306
 Training accuracy: 0.826
 Test accuracy: 0.696
 Test ROC_AUC: 0.800
Epoch: 550/600 ,learning_rate:0.003833,cost: 0.237276182
 Training accuracy: 0.848
 Test accuracy: 0.696
 Test ROC_AUC